In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import seaborn as sns
from matplotlib import pyplot as plt
import sklearn.cross_validation
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
#because dataset is skewed we can try out logistic regression with balanced weights
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix

/Library/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
%pylab inline

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.cross_validation import train_test_split

try:
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.pipeline import make_pipeline
except ImportError:
    # use backports for sklearn 1.4
    # available from https://s3.amazonaws.com/datarobotblog/notebooks/sklearn_backports.py
    from sklearn_backports import PolynomialFeatures
    from sklearn_backports import make_pipeline

# ignore DeprecateWarnings by sklearn
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Import SciKit Learn functions
from sklearn.metrics import roc_curve, auc , roc_auc_score, confusion_matrix, mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn import grid_search
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import seaborn as sns
from matplotlib import pyplot as plt
import sklearn.cross_validation
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
#because dataset is skewed we can try out logistic regression with balanced weights
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc , roc_auc_score, confusion_matrix, mean_absolute_error
from sklearn.tree import DecisionTreeClassifier , export_graphviz
from sklearn.cross_validation import train_test_split
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier 

In [4]:
df_train=pd.read_csv('train_ireg_amd_dmd.csv')
df_test=pd.read_csv('test_ireg_amd_dmd.csv')

In [5]:
df_train.head(10)

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0,1,0.766127,45,2,0.802982,9120.000000,13,0,6,0,2.0
1,1,0,0.957151,40,0,0.121876,2600.000000,4,0,0,0,1.0
2,2,0,0.658180,38,1,0.085113,3042.000000,2,1,0,0,0.0
3,3,0,0.233810,30,0,0.036050,3300.000000,5,0,0,0,0.0
4,4,0,0.907239,49,1,0.024926,63588.000000,7,0,1,0,0.0
5,5,0,0.213179,74,0,0.375607,3500.000000,3,0,1,0,1.0
6,6,0,0.305682,57,0,5710.000000,8738.957438,8,0,3,0,0.0
7,7,0,0.754464,39,0,0.209940,3500.000000,8,0,0,0,0.0
8,8,0,0.116951,27,0,46.000000,3045.627787,2,0,0,0,1.0
9,9,0,0.189169,57,0,0.606291,23684.000000,9,0,4,0,2.0


In [6]:
#Currently getting subset of data and just wrangling around and trying

Y_train=np.array(df_train.SeriousDlqin2yrs)
#features = ['RevolvingUtilizationOfUnsecuredLines', 'DebtRatio',
#            'MonthlyIncome', 'age', 'NumberOfTimes90DaysLate']
features=['RevolvingUtilizationOfUnsecuredLines','age','NumberOfTime30-59DaysPastDueNotWorse','DebtRatio','MonthlyIncome','NumberOfOpenCreditLinesAndLoans','NumberOfTimes90DaysLate','NumberRealEstateLoansOrLines','NumberOfTime60-89DaysPastDueNotWorse','NumberOfDependents']
X_train=np.array(df_train[features])

In [9]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(df_train.SeriousDlqin2yrs))
    plt.xticks(tick_marks, df_train.SeriousDlqin2yrs, rotation=45)
    plt.yticks(tick_marks, df_train.SeriousDlqin2yrs)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [7]:
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=0.25)

In [8]:
RandomForestModel = RandomForestClassifier(criterion='gini', oob_score=True ,n_estimators=3000,max_features='auto',n_jobs=-1) 
RandomForestModel.fit(X_train,Y_train)
rf_predV=RandomForestModel.predict(X_validation)
prob=RandomForestModel.predict_proba(X_validation)

In [72]:
rf_predictions=prob[:,1]
probt=RandomForestModel.predict_proba(X_train)
predT=RandomForestModel.predict(X_train)

In [73]:
#X_train_stacked=np.column_stack((X_train,probt[:,1]))
X_train_stacked=np.column_stack((X_train,predT))

In [74]:
#X_validation_stacked=np.column_stack((X_validation,rf_predictions))
X_validation_stacked=np.column_stack((X_validation,rf_predV))

In [34]:
print X_train_stacked.shape

(112500, 11)


In [ ]:
preds = prob[:,1]
fpr, tpr, threshold = roc_curve(Y_validation, preds)
roc_auc = auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [75]:
DecisionTreeModel = DecisionTreeClassifier(max_depth=6)
DecisionTreeModel=DecisionTreeModel.fit(X_train,Y_train)
dt_predV=DecisionTreeModel.predict(X_validation)
prob=DecisionTreeModel.predict_proba(X_validation)

In [76]:
dt_predictions=prob[:,1]
probt=DecisionTreeModel.predict_proba(X_train)
predT=DecisionTreeModel.predict(X_train)

In [77]:
#X_train_stacked=np.column_stack((X_train_stacked,probt[:,1]))
X_train_stacked=np.column_stack((X_train_stacked,predT))

In [78]:
#X_validation_stacked=np.column_stack((X_validation_stacked,dt_predictions))
X_validation_stacked=np.column_stack((X_validation_stacked,dt_predV))

In [79]:
#need cross validation to decide
LogisticModel = LogisticRegressionCV(Cs=[100],cv=5,penalty='l1',solver='liblinear',scoring='roc_auc') 
LogisticModel.fit(X_train,Y_train)
logistic_predV=LogisticModel.predict(X_validation)
prob=LogisticModel.predict_proba(X_validation)

In [80]:
logistic_predictions=prob[:,1]
probt=LogisticModel.predict_proba(X_train)
predT=DecisionTreeModel.predict(X_train)

In [81]:
#X_train_stacked=np.column_stack((X_train_stacked,probt[:,1]))
X_train_stacked=np.column_stack((X_train_stacked,predT))

In [82]:
#X_validation_stacked=np.column_stack((X_validation_stacked,logistic_predictions))
X_validation_stacked=np.column_stack((X_validation_stacked,logistic_predV))

In [83]:
from sklearn.neighbors import KNeighborsClassifier

NearestNeighborModel = KNeighborsClassifier(n_neighbors=13)
NearestNeighborModel.fit(X_train, Y_train)
nearest_predV=NearestNeighborModel.predict(X_validation)
prob=NearestNeighborModel.predict_proba(X_validation)

In [84]:
nearest_predictions=prob[:,1]
probt=NearestNeighborModel.predict_proba(X_train)
predT=NearestNeighborModel.predict(X_train)

In [85]:
#X_train_stacked=np.column_stack((X_train_stacked,probt[:,1]))
X_train_stacked=np.column_stack((X_train_stacked,predT))

In [86]:
#X_validation_stacked=np.column_stack((X_validation_stacked,nearest_predictions))
X_validation_stacked=np.column_stack((X_validation_stacked,nearest_predV))

In [87]:
#remaining
from sklearn.naive_bayes import GaussianNB
NaiveBayesModel = GaussianNB()
NaiveBayesModel.fit(X_train,Y_train)
naive_predV=NaiveBayesModel.predict(X_validation)
prob=NaiveBayesModel.predict_proba(X_validation)

In [88]:
naive_predictions=prob[:,1]
probt=NaiveBayesModel.predict_proba(X_train)
predT=NaiveBayesModel.predict(X_train)

In [89]:
X_train_stacked=np.column_stack((X_train_stacked,predT))

In [90]:
X_validation_stacked=np.column_stack((X_validation_stacked,naive_predV))

In [91]:
GradientBoostingtreeModel = GradientBoostingClassifier(loss='deviance', max_features = 'auto',learning_rate=0.01,
                                    n_estimators=2500,max_depth=3)
GradientBoostingtreeModel.fit(X_train,Y_train)
gb_predV=GradientBoostingtreeModel.predict(X_validation)
prob=GradientBoostingtreeModel.predict_proba(X_validation)

In [92]:
gb_predictions=prob[:,1]
probt=GradientBoostingtreeModel.predict_proba(X_train)
predT=GradientBoostingtreeModel.predict(X_train)

In [93]:
#X_train_stacked=np.column_stack((X_train_stacked,probt[:,1]))
X_train_stacked=np.column_stack((X_train_stacked,predT))

In [94]:
#X_validation_stacked=np.column_stack((X_validation_stacked,gb_predictions))
X_validation_stacked=np.column_stack((X_validation_stacked,gb_predV))

In [95]:
print X_train_stacked.shape
print X_validation_stacked.shape

(112500, 16)
(37500, 16)


In [96]:
#GradientBoostingtreeModelFinal = GradientBoostingClassifier(loss='deviance', max_features = 'auto',learning_rate=0.01,
#                                    n_estimators=2500,max_depth=3)
RandomForestModelFinal = RandomForestClassifier(criterion='entropy', oob_score=True ,n_estimators=10000,max_features='auto',n_jobs=-1,) 


In [97]:
RandomForestModelFinal.fit(X_train_stacked,Y_train)
prob=RandomForestModelFinal.predict_proba(X_validation_stacked)
#GradientBoostingtreeModelFinal.fit(X_train_stacked,Y_train)
#prob=GradientBoostingtreeModel.predict_proba(X_validation_stacked)
#print X_train_stacked.shape
#print Y_train.shape
#print X_validation_stacked.shape

In [98]:
print X_validation_stacked.shape

(37500, 16)


In [99]:
probs=RandomForestModelFinal .predict_proba(X_validation_stacked)
auc_score=roc_auc_score(Y_validation,probs[:,1])
print "ROC score on model with depth  ",roc_auc_score(Y_validation,probs[:,1])

ROC score on model with depth   0.820732663606
